In [ ]:
import keras
from tensorflow.keras.applications import ResNet50V2
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/AIDER.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

dataGenerator = keras.preprocessing.image.ImageDataGenerator(validation_split = 0.3, samplewise_center = True)

test = dataGenerator.flow_from_directory("/tmp/AIDER", class_mode='categorical', target_size = (256,256), batch_size=28, subset="validation", shuffle=True)
train = dataGenerator.flow_from_directory("/tmp/AIDER", class_mode='categorical', target_size = (256,256), batch_size=28, subset="training", shuffle=True)

Found 1626 images belonging to 3 classes.
Found 3796 images belonging to 3 classes.


In [ ]:
def disasterNet():
  
    model = keras.models.Sequential()
    #model.add(keras.layers.experimental.preprocessing.Normalization())
    model.add(keras.layers.experimental.preprocessing.RandomRotation(1,input_shape = (256,256,3)))
    model.add(keras.layers.experimental.preprocessing.RandomFlip("horizontal"))
    model.add(keras.layers.experimental.preprocessing.RandomFlip("vertical"))
    model.add(keras.layers.experimental.preprocessing.RandomZoom(0.2))
    model.add(keras.layers.experimental.preprocessing.RandomTranslation(height_factor = 0.1, width_factor = 0.1))
    model.add(ResNet50V2(include_top = False, weights = "imagenet", input_tensor = keras.Input(shape = (256, 256, 3))))
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(256, activation = "relu"))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(256, activation = "relu"))
    model.add(keras.layers.Dropout(0.6))
    model.add(keras.layers.Dense(3, activation = "softmax"))

    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate = 0.001), metrics=["categorical_accuracy"])
    
    return(model)

from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

model = disasterNet()
model.summary()
#model.fit(train, epochs=100, validation_data=test, class_weight = {0:1, 1:5, 2:1})
model.fit(train, epochs=100, validation_data=test, callbacks=[mcp_save, reduce_lr_loss])

#IMPORTANT: Your model must be saved as "Q3.h5"
model.save("./Q3.h5")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_rotation_2 (RandomRot (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip_4 (RandomFlip)   (None, 256, 256, 3)       0         
_________________________________________________________________
random_flip_5 (RandomFlip)   (None, 256, 256, 3)       0         
_________________________________________________________________
random_zoom_2 (RandomZoom)   (None, 256, 256, 3)       0         
_________________________________________________________________
random_translation_2 (Random (None, 256, 256, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 8, 8, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)             